In [ ]:
from utils_general import *
from utils_methods import *
import json

# from utils_methods_FedDC import train_FedDC

In [ ]:
# # Set the GPU you want to use
# torch.cuda.set_device(0)  # This line sets the second GPU as the device.

# # Now to check whether the correct device is set
# print(torch.cuda.current_device()) 

In [ ]:
class config(object):
    def __init__(self):
        self.dataset="cifar10"
        self.device = "cuda"
        self.comm_round=2
        self.lr = 0.01
        self.batch_size = 64
        self.epochs = 1
        self.weight_decay = 1e-3
        self.n_parties = 5
        self.seed = 43
        self.alg="fedavg"
        self.rootdir="./result/bench2/"
        self.datadir="./Folder/"
        self.beta=0.1
        self.model = 'cifar10_LeNet' # Model type


args = config()

In [ ]:
from utils_libs_wy import get_logger

# def get_logger(logger_path):
#     logging.basicConfig(
#         filename=logger_path,
#         # filename='/home/qinbin/test.log',
#         format='[%(asctime)s] %(levelname)s: %(message)s',
#         datefmt='%m-%d %H:%M', 
#         level=logging.DEBUG, 
#         filemode='w'
#     )

#     logger = logging.getLogger()
#     logger.setLevel(logging.INFO)
#     ch = logging.StreamHandler()
#     logger.addHandler(ch)

#     return logger 

In [ ]:
seed = args.seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)

In [ ]:
# # Dataset initialization
# data_path = args.datadir
# n_client = args.n_parties
# # Dirichlet (0.3)

# data_obj = DatasetObject(dataset=args.dataset.upper(), n_client=n_client, seed=args.seed, unbalanced_sgm=0, rule='Drichlet', rule_arg=args.beta, data_path=data_path)


In [ ]:
# # Model function
# model_func = lambda : client_model(args.model)
# init_model = model_func()



tag = f"{args.dataset}-{args.model}-{args.alg}-N{args.n_parties}-beta{args.beta}-ep{args.epochs}-lr{args.lr}-round{args.comm_round}"
args.exp_dir = os.path.join(args.rootdir, tag) 
args.logdir = os.path.join(args.exp_dir, f"seed{args.seed}")
os.makedirs(args.logdir)

argument_path='experiment_arguments-%s.json' % datetime.datetime.now().strftime("%Y-%m-%d-%H:%M-%S")
with open(os.path.join(args.logdir, argument_path), 'w') as f:
    json.dump(str(args), f)

log_path= 'experiment_log-%s' % (datetime.datetime.now().strftime("%Y-%m-%d-%H:%M-%S")) + '.log'
logger = get_logger(logger_path=os.path.join(args.logdir, log_path))

# load and partition dataset
ds, data_info = get_dataset(dataset=args.dataset)  
split_distribution, dataidx_map_train = partition_labeldir(
    targets=ds['train_labels'], 
    num_classes=data_info['num_classes'], 
    n_parties=args.n_parties,
    beta=args.beta
)
client_data_train = make_client_dataset_from_partition(ds['train_data'], args.n_parties, dataidx_map_train)  
logger.info('Client dataset partitioning completed')
traindata_cls_counts = record_net_data_stats(ds['train_labels'], dataidx_map_train, logger)

model_func = lambda : ConvNet(
    channel=data_info['channel'], 
    num_classes=data_info['num_classes'], 
    net_width=128, 
    net_depth=3, 
    net_act="relu", 
    net_norm="batchnorm", 
    net_pooling="avgpooling", 
    im_size=data_info['img_size']
)
init_model = model_func()
# init_model.to(args.device)
# local_trainloaders = (DataLoader(client_data_train[i], batch_size=args.batch_size, shuffle=True) for i in range(args.n_parties))
# central_testloader = DataLoader(ds["test_data"], batch_size=128, shuffle=False)

  

args.device="mps"

In [ ]:
# print(f'Trying {args.alg}')
# train_FedAvg(
#     args,
#     logger,
#     init_model,
#     client_data_train,
#     ds["test_data"]
# )    

In [ ]:
args.alg = "feddc"
args.alpha = 1e-2
# args.lr=0.1
n_data_per_client_avg = len(ds["train_data"]) / args.n_parties
n_iter_per_epoch  = np.ceil(n_data_per_client_avg/args.batch_size)
n_minibatch = (args.epochs*n_iter_per_epoch).astype(np.int64)

from utils_methods_wy import train_FedDC_wy
print(f'Trying {args.alg}')
train_FedDC_wy(
    args,
    logger,
    init_model,
    client_data_train,
    ds["test_data"],
    n_minibatch,
    args.alpha
)    

In [ ]:
# clnt_mdl = copy.deepcopy(init_model.to(args.device))

In [ ]:
# get_mdl_params(clnt_mdl)